In [18]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db"  # nombre del servicio en docker-compose
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

# Leer una tabla completa
df = pd.read_sql("SELECT * FROM grado;", engine)
df.head()

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
1,2,Antropología Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
2,3,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...
3,4,Conservación y Restauración del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...
4,5,Artes Escénicas,2,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...


In [19]:
df.shape

(229, 5)

In [20]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': '9c8fd69b1610', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'zUwZFeuWRpi79BScYMuASA', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [21]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)

In [22]:
mapping = {
    "mappings": {
        "properties": {
            "id": { "type": "integer" },        
            "nombre": { "type": "keyword" },     
            "descripcion": { "type": "text" },
            "id_area": { "type": "integer" },
            "salidas": { "type": "text" },
        }
    }
}

index_name = 'salidas'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, mappings=mapping["mappings"])
print(f"Index '{index_name}' created with mapping.")

Index 'salidas' created with mapping.


In [23]:
import json
from elasticsearch.helpers import bulk

def bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for doc in batch:
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

salidas = df.to_dict(orient='records')
bulk_index_data(es, salidas, index_name)


Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 29 Errors: []


In [31]:
import json

# Query 1

res = es.search(
    index="salidas",
    size=3,
    query={
        "term": {
            "nombre": "Bellas Artes"
        }
    }
)
print(json.dumps(res.body, indent=4))

{
    "took": 24,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 1,
            "relation": "eq"
        },
        "max_score": 5.032614,
        "hits": [
            {
                "_index": "salidas",
                "_id": "3",
                "_score": 5.032614,
                "_source": {
                    "id": 3,
                    "nombre": "Bellas Artes",
                    "id_area": 2,
                    "descripcion": "Los estudios de Bellas Artes tienen entre sus objetivos fundamentales la conservaci\u00f3n y expansi\u00f3n del patrimonio art\u00edsticocultural, la educaci\u00f3n est\u00e9tica y la formaci\u00f3n t\u00e9cni cocient\u00edfica del alumno en el campo profe sional del arte puro, de la est\u00e9tica aplicada o de la docencia gr\u00e1ficopl\u00e1stica. El Grado en Bellas Artes supone, para el que la es tudia y l

In [30]:
# Query
res = es.search(
    index="salidas",
    size=3,
    query={
        "match": {
            "salidas": "profesor"
        }
    }
)
print(json.dumps(res.body, indent=4))

{
    "took": 37,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 10,
            "relation": "eq"
        },
        "max_score": 5.012066,
        "hits": [
            {
                "_index": "salidas",
                "_id": "48",
                "_score": 5.012066,
                "_source": {
                    "id": 48,
                    "nombre": "Lenguas Modernas y sus Literaturas",
                    "id_area": 12,
                    "descripcion": "La existencia en la UCM de las filolog\u00edas mo dernas cuenta ya con m\u00e1s de medio siglo y tienen una s\u00f3lida reputaci\u00f3n nacional e inter nacional, como demuestran los numerosos convenios con las principales universidades nacionales y extranjeras, que han dado lugar a la creaci\u00f3n de numerosas redes de investi gaci\u00f3n. Las transformaciones sociales deriva das d

In [25]:
# indice semántico

vector_index_mapping = {
    "properties": {
        "id": { "type": "integer" },        
        "nombre": { "type": "keyword" },     
        "id_area": { "type": "integer" },
        "salidas": { "type": "text" },
        "descripcion": { "type": "text" },
        "descripcion_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionalityof the embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

vector_index_name = "vector_descripcion"
if es.indices.exists(index=vector_index_name):
    es.indices.delete(index=vector_index_name)
es.indices.create(
    index=vector_index_name,
    mappings=vector_index_mapping
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector_descripcion'})

In [26]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for the sentences
embeddings = model.encode(df["descripcion"].values.tolist())

# Print the vector for the first sentence
print(embeddings[0])

[ 4.52831239e-02  1.22150369e-01 -8.74836892e-02  6.78779334e-02
 -2.36771740e-02 -2.62602679e-02  3.67369642e-03 -3.03924363e-03
  7.08474442e-02  1.65133610e-01  7.43214041e-02 -1.06290810e-01
 -5.17418757e-02  4.67618257e-02  6.69129118e-02 -5.53936418e-03
 -8.59947875e-02 -4.16934416e-02 -3.49510461e-02  2.64398083e-02
  3.74254733e-02  1.31367054e-02 -1.45175271e-02 -2.27634832e-02
 -1.55368716e-01 -8.09538513e-02  1.95590556e-02 -1.39014870e-01
  2.76912302e-02 -2.30619833e-02 -6.25008577e-03  4.24942821e-02
  1.17510632e-01  1.48596376e-01 -1.70421809e-01  9.18175727e-02
  8.41074884e-02  8.83894265e-02  1.32022947e-01  1.44034237e-01
  4.33059130e-03 -4.34244312e-02  1.42072693e-01  5.75205032e-03
 -7.02844486e-02  9.58932796e-04  6.73831673e-05  7.57784173e-02
 -1.67612955e-01 -9.23196673e-02 -5.70666194e-02 -5.74748889e-02
 -9.89328176e-02  5.48897684e-03 -4.48590470e-03  5.17457761e-02
  3.13876830e-02 -4.97543924e-02 -4.94946986e-02 -6.08411431e-03
  1.46229222e-01 -3.70180

In [27]:
import json
from elasticsearch.helpers import bulk

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, vector_index_name)

Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 29 Errors: []


In [29]:
# Query semantica

query_sentence = "análisis de datos"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":5,
     "num_candidates":100
}
res = es.search(
    index=vector_index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")
    

Search Results:
Document ID: 84, Nombre Título: Análisis de Negocios, Description: Este grado se centra en el análisis de datos masivos en el ámbito de los negocios que permitan establecer relaciones significativas. Es una carrera bilingüe de alto contenido tecnológico formada por tres grandes pilares: . Administración y Dirección de Empresas, de análisis estadístico y de informática, con materias tales como Management Analytics, Data Mining, Business Intelligence, Big Data, Matemáticas, Contabilidad y Auditoría. . Derecho, Fiscalidad, Economía, Humani dades en la Era Digital. . Informática aplicada al Business Analytics y Business English. Algunas de las competencias básicas y gene rales a adquirir por el alumno: Organización, sistematización y planificación en la identificación de problemas, pautas y modelos en el contexto del big data. Análisis de datos a gran escala procedentes de fuentes audiovisuales, textos y numéricas. Capacidad para el cumplimiento de obje tivos, resolución de